In [1]:
import sys
sys.path.append('/Users/katiehuang/Desktop/metis/projects/onl_ds5_project_2/py')
from loadpage import *
import importlib
from fictiondb import *

In [2]:
from bs4 import BeautifulSoup
import requests
import time, os

In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [4]:
# # Open automated Chrome with fictionDB webpage
# driver = webdriver.Chrome(chromedriver)
# driver.get("https://www.fictiondb.com/search/search.htm")

# # Search for author (test with Jane Austen)
# search_bar = driver.find_element_by_xpath("//input[@name='author'][@type='text']")
# search_bar.clear()
# search_bar.send_keys("Jane Austen")
# search_bar.send_keys(Keys.RETURN)

# # Click on the author link
# elem = driver.find_element_by_link_text("Austen, Jane")
# elem.click()

# # Find publications and create list of year
# publication = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")
# years = [int(s.text) for s in publication]

# years

In [5]:
# may be useful??
# WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@class='btn btn-primary' and @id='YesBtn']"))).click()

In [6]:
# driver.find_element_by_xpath(("//html")).click()

In [14]:
author_dict = {}

In [15]:
authors = ["Jane Austen","J.K. Rowling"]

In [18]:
for author in authors:
    years = search(author)
    author_dict[author] = years

In [19]:
author_dict

{'Jane Austen': [1818,
  1817,
  1815,
  1814,
  1813,
  1811,
  1805,
  1805,
  1803,
  1790,
  1786],
 'J.K. Rowling': [2020,
  2018,
  2016,
  2016,
  2016,
  2016,
  2012,
  2008,
  2007,
  2005,
  2003,
  2001,
  2001,
  2000,
  1999,
  1999,
  1998]}

In [9]:
# search("J.K. Rowling")

In [10]:
# find_years()

In [11]:
# publication = driver.find_elements_by_xpath("//a[@itemprop='datePublished']")

### Selenium on goodreads

Need to search on goodreads using movie title to find the book match.

In [ ]:
driver = webdriver.Chrome(chromedriver)

In [ ]:
driver.get("https://www.goodreads.com/")

In [ ]:
# Search for movie title
# Take After we collided for example
search_bar = driver.find_element_by_xpath("//input[@name='query'][@type='text']")
search_bar.clear()
search_bar.send_keys("After we collided")
search_bar.send_keys(Keys.RETURN)

In [ ]:
# get ad page and reactid
ad = driver.page_source
ad_soup = BeautifulSoup(ad, "lxml")
button = ad_soup.find_all('button',class_="gr-iconButton")[-1]
rid = (button.get('data-reactid'))

# Click on the cross button
cross = driver.find_element_by_xpath("//button[@data-reactid='" + rid + "'][@type='button']")
cross.click()

In [ ]:
# Click on the first results
elem = driver.find_element_by_xpath("//a[@class='bookTitle']")
elem.click()

In [ ]:
book = BeautifulSoup(driver.page_source, "lxml")
get_bookdata(book)

In [ ]:
# Refine the function

def search_goodreads(movie):
    driver.get("https://www.goodreads.com/")
    
    # Set book default
    book = {'book_title': np.nan,
 'author': np.nan,
 'rating_value': np.nan,
 'rating_count': np.nan,
 'review_count': np.nan,
 'page': np.nan,
 'year': np.nan}

    # Search movie title
    search_bar = driver.find_element_by_xpath("//input[@name='query'][@type='text']")
    search_bar.clear()
    search_bar.send_keys(movie)
    search_bar.send_keys(Keys.RETURN)
    time.sleep(5)

    actions = ActionChains(driver)
    actions.send_keys(Keys.RETURN)
    actions.perform()
    # Click on the first result
    try:
        elem = driver.find_element_by_class_name("bookTitle").click()
        # Make soup and get data
        book_soup = BeautifulSoup(driver.page_source, "lxml")
        book = get_bookdata(book_soup)
    except:
        pass
    

    return book

In [ ]:
# Test on short list of movies missing books
found_book1=[]
driver = webdriver.Chrome(chromedriver)
for movie in ['After We Collided','Mowgli Legend of the Jungle','The Beastmaster']:
    result = search_goodreads(movie)
    found_book1.append(result)

In [ ]:
found_book1

### Create a found_book list of data for books found by searching movie titles

In [ ]:
# Load in movie_data picke file
movie_df = pd.read_pickle('../dump/movie_data')
book_df = pd.read_pickle('../dump/book_data')

In [ ]:
movie_list = list(movie_df.movie_title)
book_list = list(book_df.book_title)

In [ ]:
need_book_pre = list(set(movie_list).difference(set(book_list)))
need_book = [book.replace('&','and').replace(":",'') for book in need_book_pre]

In [ ]:
len(need_book)

In [ ]:
found_book=[]
driver = webdriver.Chrome(chromedriver)
for movie in need_book:
    result = search_goodreads(movie)
    found_book.append(result)

In [ ]:
# Save the list to pd.DataFrame
found_book_df = pd.DataFrame(found_book)
# Save the data to pickle file
found_book_df.to_pickle('../dump/found_book')

In [ ]:
len(found_book)

In [ ]:
# Try loading from picke files
found_book_df = pd.read_pickle('../dump/found_book')

In [ ]:
found_book_df